In [ ]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from tuning_curves_functions import get_tc, get_odd_firing_idx, linearize

import info.R063d2_info as r063d2
import info.R063d3_info as r063d3
import info.R063d4_info as r063d4
import info.R063d5_info as r063d5
import info.R063d6_info as r063d6
import info.R066d1_info as r066d1
import info.R066d2_info as r066d2
import info.R066d4_info as r066d4

In [ ]:
pickle_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\python-vdmlab\\projects\\emily_shortcut\\plots\\sequence\\'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\python-vdmlab\\projects\\emily_shortcut\\plots\\sequence\\'

In [ ]:
info = r063d4

In [ ]:
print(info.session_id)
pos = info.get_pos(info.pxl_to_cm)
csc = info.get_csc()
spikes = info.get_spikes()

tc = get_tc(info, pos, pickle_filepath)

heatmap_filename = info.session_id + '_spike_heatmaps.pkl'
pickled_spike_heatmaps = os.path.join(pickle_filepath, heatmap_filename)
if os.path.isfile(pickled_spike_heatmaps):
    with open(pickled_spike_heatmaps, 'rb') as fileobj:
        spike_heatmaps = pickle.load(fileobj)
else:
    spikes = info.get_spikes()

    all_neurons = list(range(1, len(spikes['time'])))
    spike_heatmaps = vdm.get_heatmaps(all_neurons, spikes, pos)
    with open(pickled_spike_heatmaps, 'wb') as fileobj:
        pickle.dump(spike_heatmaps, fileobj)

swr_times, swr_idx, filtered_butter = vdm.detect_swr_hilbert(csc, fs=info.fs)




all_u_fields = vdm.find_fields(tc['u'])
all_shortcut_fields = vdm.find_fields(tc['shortcut'])
all_novel_fields = vdm.find_fields(tc['novel'])

# u_compare = vdm.find_fields(tc['u'], hz_thres=3)
# shortcut_compare = vdm.find_fields(tc['shortcut'], hz_thres=3)
# novel_compare = vdm.find_fields(tc['novel'], hz_thres=3)

# u_fields_unique = vdm.unique_fields(all_u_fields, shortcut_compare, novel_compare)
# shortcut_fields_unique = vdm.unique_fields(all_shortcut_fields, u_compare, novel_compare)
# novel_fields_unique = vdm.unique_fields(all_novel_fields, u_compare, shortcut_compare)

u_fields_size = vdm.sized_fields(all_u_fields, min_length=3, max_length=15)
shortcut_fields_size = vdm.sized_fields(all_shortcut_fields, min_length=3, max_length=15)
novel_fields_size = vdm.sized_fields(all_novel_fields, min_length=3, max_length=15)

u_fields = vdm.get_single_field(u_fields_size)
shortcut_fields = vdm.get_single_field(shortcut_fields_size)
novel_fields = vdm.get_single_field(novel_fields_size)




In [ ]:
this_tc = tc['u']

sort_idx = vdm.get_sort_idx(this_tc)

odd_firing_idx = get_odd_firing_idx(this_tc)

these_fields = []
for key in u_fields:
    these_fields.append(key)

field_spikes = []
field_tc = []
for idx in sort_idx:
    if idx not in odd_firing_idx:
        if idx in these_fields:
            field_spikes.append(spikes['time'][idx])
            field_tc.append(this_tc[idx])

In [ ]:
t_start = info.task_times['prerecord'][0]
t_stop = info.task_times['postrecord'][1]
linear, zone = linearize(info, pos, t_start, t_stop)

In [ ]:
this_linear = linear['u']

In [ ]:
print(len(these_fields))

In [ ]:
colours = ['#bd0026', '#fc4e2a', '#ef3b2c', '#ec7014', '#fe9929', 
           '#78c679', '#41ab5d', '#238443', '#66c2a4', '#41b6c4', 
           '#1d91c0', '#8c6bb1', '#225ea8', '#88419d', '#ae017e', 
           '#dd3497', '#f768a1', '#fcbba1', '#fc9272', '#fb6a4a', 
           '#e31a1c', '#fb6a4a', '#993404', '#b30000', '#800026',
           '#bd0026', '#fc4e2a', '#fb6a4a', '#ef3b2c', '#ec7014', 
           '#fe9929', '#78c679', '#41ab5d', '#238443', '#66c2a4', 
           '#41b6c4', '#1d91c0', '#8c6bb1', '#225ea8', '#88419d', 
           '#ae017e', '#dd3497', '#f768a1', '#fcbba1', '#fc9272', 
           '#fb6a4a', '#e31a1c', '#fb6a4a', '#993404', '#b30000', 
           '#800026', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k', 'k']

In [ ]:
sns.set_style('white')
sns.set_style('ticks')

In [ ]:
print(swr_times['start'][-1])

In [ ]:
print(info.task_times['postrecord'][1])

In [ ]:
swr = 1991
# for swr in [2090, 2196]:

print(swr)
ms = 10
loc = 1

# start_time = info.task_times['phase3'][0]
# stop_time = info.task_times['phase3'][0]+30

# start_time_swr = info.task_times['postrecord'][0]
# stop_time_swr = info.task_times['postrecord'][0]+1

start_time = info.sequence['u']['run_start'][1]
stop_time = info.sequence['u']['run_stop'][1]

# start_time_swr = 9024.7
# stop_time_swr = 9025.3

# start_time_swr = 9040.05
# stop_time_swr = 9040.65

start_time_swr = info.sequence['u']['swr_start'][1]+0.2
stop_time_swr = info.sequence['u']['swr_start'][1]+0.6



# for i, (start_time, stop_time, start_time_swr, stop_time_swr) in enumerate(zip(info.sequence['run_start'], 
#                                                                                info.sequence['run_stop'], 
#                                                                                info.sequence['swr_start'], 
#                                                                                info.sequence['swr_stop'])):
spike_loc = 2

rows = len(field_spikes)+1
cols = 7
fig = plt.figure()
ax1 = plt.subplot2grid((rows, cols), (0, 1), rowspan=rows, colspan=4)
ax2 = plt.subplot2grid((rows, cols), (0, 5), rowspan=rows, colspan=2)

max_position = np.zeros(len(this_linear['time']))
max_position.fill(np.max(this_linear['position']/90))
ax1.plot(this_linear['time'], max_position, color='#bdbdbd', lw=1)
ax1.plot(this_linear['time'], this_linear['position']/90, 'k', lw=1)
for idx, neuron_spikes in enumerate(field_spikes):
    ax1.plot(neuron_spikes, np.ones(len(neuron_spikes))+(idx*spike_loc+3), '|', 
             color=colours[int(np.floor((idx*spike_loc+3)/spike_loc))], ms=ms, mew=2)
ax1.set_xlim([start_time, stop_time])
ax1.set_ylim([0, len(field_spikes)*spike_loc+3])
vdm.add_scalebar(ax1, matchy=False, loc=loc)
# plt.setp(ax1, xticks=[], xticklabels=[], yticks=[])

plt.plot(csc['time'], csc['data']*1000+1.25, 'k', lw=1)
plt.plot(csc['time'], filtered_butter*1000+0.5, 'b', lw=1)
for j, neuron_spikes in enumerate(field_spikes):
    ax2.plot(neuron_spikes, np.ones(len(neuron_spikes))+(j*spike_loc+3), '|', 
             color=colours[int(np.floor((j*spike_loc+3)/spike_loc))], ms=ms, mew=2)
ax2.set_xlim([start_time_swr, stop_time_swr])
ax2.set_ylim([0, len(field_spikes)*spike_loc+3])
vdm.add_scalebar(ax2, matchy=False, loc=loc)
# plt.setp(ax2, xticks=[], xticklabels=[], yticks=[])

x = list(range(0, len(field_tc[0])))

for ax_loc in range(0, rows-1):
    ax = plt.subplot2grid((rows, cols), (ax_loc, 0))

    idx = rows - ax_loc - 1
    ax.plot(field_tc[idx-1], color=colours[idx-1])
    ax.fill_between(x, 0, field_tc[idx-1], facecolor=colours[idx-1])
    max_loc = np.where(field_tc[idx-1] == np.max(field_tc[idx-1]))[0][0]
    ax.text(max_loc-3, 1, str(int(np.ceil(np.max(field_tc[idx-1])))))
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.setp(ax, xticks=[], xticklabels=[], yticks=[])

sns.despine()
plt.show()
#     filename = info.session_id + '_sequence-swr' + str(i) + '.png'
#     print(filename)
#     savepath = os.path.join(output_filepath, filename)
#     plt.savefig(savepath, dpi=300, bbox_inches='tight')
#     plt.close()

In [ ]:
4882+30

In [ ]:
max_position = np.zeros(len(linear['u']['time']))
max_position.fill(np.max(linear['u']['position']/90))
plt.plot(linear['u']['time'], max_position, color='#969696', lw=1)
plt.plot(linear['u']['time'], np.array(linear['u']['position'])/90, 'k')
plt.xlim(start_time, stop_time)
plt.show()

In [ ]:
max_position = np.zeros(len(linear['u']['time']))
max_position.fill(np.max(linear['u']['position']/90))
print(max_position)

In [ ]:
plt.plot(csc['time'], csc['data']*1000+1.25, 'b')
plt.plot(csc['time'], filtered_butter*1000+0.5, 'b')
plt.xlim(start_time_swr, stop_time_swr)
plt.ylim(0, 2)
plt.show()


In [ ]:
print('run:', start_time, stop_time)
print('swr:', start_time_swr, stop_time_swr)

In [ ]:
print(np.where(field_tc[idx-1] == np.max(field_tc[idx-1]))[0][0])

In [ ]:
14609.0 - 14579.0

In [ ]:
ordered_spikes = []
for idx in sort_idx:
    if idx not in odd_firing_idx:
        ordered_spikes.append(spikes['time'][idx])

In [ ]:
all_neurons = []
for neuron_spikes in field_spikes:
    this_neuron = []
    for swr_start, swr_stop in zip(swr_times['start'], swr_times['stop']):
        start_idx = vdm.find_nearest_idx(neuron_spikes, swr_start)
        stop_idx = vdm.find_nearest_idx(neuron_spikes, swr_stop)
        this_neuron.append(len(neuron_spikes[start_idx:stop_idx]))
    all_neurons.append(this_neuron)

In [ ]:
count_mult_neurons = []
for i, swr_start in enumerate(swr_times['start']):
    count_single_neurons = []
    for swr_spike_num in all_neurons:
        if swr_spike_num[i] > 0:
            count_single_neurons.append(i)
            if len(count_single_neurons) > 2:
                if i not in count_mult_neurons:
                    count_mult_neurons.append(i)
print('Swr events with multiple neurons:', len(count_mult_neurons))
print('Number of neurons:', len(field_spikes))

In [ ]:
for get_this in count_mult_neurons:
    mult_idx = get_this
    print(mult_idx)

    plt.plot(csc['time'], csc['data']*10500)
    for swr_start, swr_stop in zip(swr_idx['start'], swr_idx['stop']):
        plt.plot(csc['time'][swr_start:swr_stop], csc['data'][swr_start:swr_stop]*10500, 'r')

    for i, neuron_spikes in enumerate(field_spikes):
        plt.plot(neuron_spikes, np.ones(len(neuron_spikes))+i*2+1, '|', color='k', ms=10, mew=1)
    plt.xlim(csc['time'][swr_idx['start'][idx]]-1, csc['time'][swr_idx['start'][idx]]+1)

    plt.ylim(-5, len(field_spikes)*2+1)
    plt.xlim(swr_times['start'][mult_idx]-0.1, swr_times['stop'][mult_idx]+0.1)
    plt.show()

In [ ]:
swr_idx['start']